In [1]:
%store -r import_all_libs

In [2]:
import_all_libs

In [3]:
from datetime import date

In [4]:
fat_ljs_atual = pd.read_csv('/home/tuts/datasets/uspvsufmg/faturamento_lojas_atuais.csv')
cen_exp = pd.read_csv('/home/tuts/datasets/uspvsufmg/cenarios_expansao.csv')
ljs_atual = pd.read_csv('/home/tuts/datasets/uspvsufmg/lojas_atuais.csv')

In [ ]:
cen_exp.head()

In [5]:
df = fat_ljs_atual.merge(ljs_atual, on='cod_loja', how='inner')
df["datetime"] = pd.to_datetime(df.ano*100 + df.mes, format='%Y%m')
df.drop(["ano", "mes"], axis=1, inplace=True)
df.head()

,cod_loja,categoria,qtde,receita,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,datetime
0,1,CATEG_01,301.00,9164.86,4125506005002,4125506,NaN,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.17,0.17,115.96,2.35,NaN,NaN,nan,nan,nan,nan,nan,nan,2016-01-01
1,1,CATEG_01,187.00,8175.55,4125506005002,4125506,NaN,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.17,0.17,115.96,2.35,NaN,NaN,nan,nan,nan,nan,nan,nan,2016-02-01
2,1,CATEG_01,242.00,10521.67,4125506005002,4125506,NaN,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.17,0.17,115.96,2.35,NaN,NaN,nan,nan,nan,nan,nan,nan,2016-03-01
3,1,CATEG_01,104.00,4560.91,4125506005002,4125506,NaN,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.17,0.17,115.96,2.35,NaN,NaN,nan,nan,nan,nan,nan,nan,2016-04-01
4,1,CATEG_01,100.00,4263.30,4125506005002,4125506,NaN,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.17,0.17,115.96,2.35,NaN,NaN,nan,nan,nan,nan,nan,nan,2016-05-01


In [6]:
s = set(fat_ljs_atual.columns) - set(['ano', 'mes'])
s.add('datetime')
s

{'categoria', 'cod_loja', 'datetime', 'qtde', 'receita'}

In [7]:
fat_mes_df = df[list(s)].groupby(["cod_loja", "datetime"] , as_index=False).sum()
fat_mes_df.head()

,cod_loja,datetime,receita,qtde
0,1,2016-01-01,441833.37,13034.00
1,1,2016-02-01,395340.22,11411.00
2,1,2016-03-01,587810.18,15030.00
3,1,2016-04-01,534618.84,12726.00
4,1,2016-05-01,595477.92,13978.00


In [8]:
df_fat_mes = fat_mes_df.merge(ljs_atual, on="cod_loja", how='inner')

In [9]:
df_fat_mes.cod_loja.nunique()

3130

In [10]:
lojas_sem_24_meses = []
for i in df_fat_mes["cod_loja"].unique():
    if df_fat_mes[df_fat_mes["cod_loja"] == i].shape[0] != 24:
        lojas_sem_24_meses.append(i)

train_df = df_fat_mes[~df_fat_mes["cod_loja"].isin(lojas_sem_24_meses).values]

In [11]:
train_df.shape[0]/24

2963.0

In [12]:
train_df.isna().sum()

cod_loja             0
datetime             0
receita              0
qtde                 0
cod_ap               0
cod_municipio        0
feature_01       45672
feature_02           0
feature_03           0
feature_04           0
feature_05           0
feature_06           0
feature_07           0
feature_08           0
feature_09           0
feature_10           0
feature_11       65352
feature_12       65352
feature_13       50376
feature_14       50376
feature_15       50376
feature_16       50376
feature_17       50376
feature_18       50376
dtype: int64

# Imputing

In [13]:
print('## FEATURE 1 ##')
print(train_df.feature_01.describe())
print('## FEATURE 11 ##')
print(train_df.feature_11.describe())
print('## FEATURE 12 ##')
print(train_df.feature_12.describe())
print('## FEATURE 13 ##')
print(train_df.feature_13.describe())
print('## FEATURE 14 ##')
print(train_df.feature_14.describe())
print('## FEATURE 15 ##')
print(train_df.feature_15.describe())
print('## FEATURE 16 ##')
print(train_df.feature_16.describe())
print('## FEATURE 17 ##')
print(train_df.feature_17.describe())
print('## FEATURE 18 ##')
print(train_df.feature_18.describe())

## FEATURE 1 ##
count                   25440
unique                      3
top       FEATURE_01_VALUE_10
freq                    14808
Name: feature_01, dtype: object
## FEATURE 11 ##
count                    5760
unique                      7
top       FEATURE_11_VALUE_02
freq                     2304
Name: feature_11, dtype: object
## FEATURE 12 ##
count                    5760
unique                      2
top       FEATURE_12_VALUE_01
freq                     3600
Name: feature_12, dtype: object
## FEATURE 13 ##
count   20736.00
mean       94.81
std         8.60
min        44.10
25%        92.20
50%       100.00
75%       100.00
max       100.00
Name: feature_13, dtype: float64
## FEATURE 14 ##
count   20736.00
mean       83.70
std        13.58
min         0.00
25%        79.68
50%        86.90
75%        91.90
max       100.00
Name: feature_14, dtype: float64
## FEATURE 15 ##
count   20736.00
mean       82.66
std        34.42
min         0.00
25%       100.00
50%       100.00
75%

In [14]:
has_na_cols = list()
for col in train_df.columns:
    if train_df[col].isnull().values.any():
        has_na_cols.append(col)
has_na_cols

['feature_01',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18']

In [15]:
for na_col in has_na_cols:
    if train_df[na_col].dtype == object:
        print(train_df[na_col].mode()[0])
        train_df[na_col].fillna(train_df[na_col].mode()[0], inplace=True)
    else:
        train_df[na_col].fillna(train_df[na_col].median(), inplace=True)
train_df.isna().sum()

FEATURE_01_VALUE_10
FEATURE_11_VALUE_02
FEATURE_12_VALUE_01



A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


cod_loja         0
datetime         0
receita          0
qtde             0
cod_ap           0
cod_municipio    0
feature_01       0
feature_02       0
feature_03       0
feature_04       0
feature_05       0
feature_06       0
feature_07       0
feature_08       0
feature_09       0
feature_10       0
feature_11       0
feature_12       0
feature_13       0
feature_14       0
feature_15       0
feature_16       0
feature_17       0
feature_18       0
dtype: int64

In [16]:
cols = train_df.columns.values.tolist()
cols

['cod_loja',
 'datetime',
 'receita',
 'qtde',
 'cod_ap',
 'cod_municipio',
 'feature_01',
 'feature_02',
 'feature_03',
 'feature_04',
 'feature_05',
 'feature_06',
 'feature_07',
 'feature_08',
 'feature_09',
 'feature_10',
 'feature_11',
 'feature_12',
 'feature_13',
 'feature_14',
 'feature_15',
 'feature_16',
 'feature_17',
 'feature_18']

In [17]:
model_df = train_df[cols]
model_df["month"] = model_df["datetime"].dt.month
model_df["quarter"] = model_df["datetime"].dt.quarter
model_df["year"] = model_df["datetime"].dt.year
model_df.head()

,cod_loja,datetime,receita,qtde,cod_ap,cod_municipio,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,month,quarter,year
0,1,2016-01-01,441833.37,13034.00,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.17,0.17,115.96,2.35,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.00,86.90,100.00,100.00,91.05,90.50,1,1,2016
1,1,2016-02-01,395340.22,11411.00,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.17,0.17,115.96,2.35,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.00,86.90,100.00,100.00,91.05,90.50,2,1,2016
2,1,2016-03-01,587810.18,15030.00,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.17,0.17,115.96,2.35,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.00,86.90,100.00,100.00,91.05,90.50,3,1,2016
3,1,2016-04-01,534618.84,12726.00,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.17,0.17,115.96,2.35,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.00,86.90,100.00,100.00,91.05,90.50,4,2,2016
4,1,2016-05-01,595477.92,13978.00,4125506005002,4125506,FEATURE_01_VALUE_10,FEATURE_02_VALUE_04,True,21.76,FEATURE_05_VALUE_04,31,0.17,0.17,115.96,2.35,FEATURE_11_VALUE_02,FEATURE_12_VALUE_01,100.00,86.90,100.00,100.00,91.05,90.50,5,2,2016


In [18]:
str_cols = list()
for col in model_df.columns.values.tolist():
    if model_df[col].dtype == object:
        str_cols.append(col)

In [19]:
str_cols.append('datetime') 
str_cols.append('receita')

In [20]:
str_cols

['feature_01',
 'feature_02',
 'feature_05',
 'feature_11',
 'feature_12',
 'datetime',
 'receita']

In [21]:
train = model_df[~model_df["datetime"].isin([date(2017,10,1), date(2017,11,1), date(2017,12,1)])]
test = model_df[model_df["datetime"].isin([date(2017,10,1), date(2017,11,1), date(2017,12,1)])]

X_train = train.drop(str_cols, axis=1)
y_train = train["receita"]

X_test = test.drop(str_cols, axis=1)
y_test = test["receita"]

# Train

In [22]:
errors = list()

In [ ]:
rf = RandomForestRegressor(n_estimators=500)

rf.fit(X_train.values, y_train.values)
y_pred = rf.predict(X_test)
m = np.sqrt(mean_squared_error(y_test, y_pred))
errors.append(m)
m

In [ ]:
lojas = ljs_atual.cod_loja.unique()

In [ ]:
has_na_cols = list()
for col in train_df.columns:
    if train_df[col].isnull().values.any():
        has_na_cols.append(col)
has_na_cols

for na_col in has_na_cols:
    if train_df[na_col].dtype == object:
        train_df[na_col].fillna(train_df[na_col].mode()[0], inplace=True)
    else:
        train_df[na_col].fillna(train_df[na_col].median(), inplace=True)
train_df.isna().sum()

cols = train_df.columns.values.tolist()

model_df = train_df[cols]
model_df["month"] = model_df["datetime"].dt.month
model_df["quarter"] = model_df["datetime"].dt.quarter
model_df["year"] = model_df["datetime"].dt.year

str_cols = list()
for col in model_df.columns.values.tolist():
    if model_df[col].dtype == object:
        str_cols.append(col)

In [ ]:
preds = []
for loja in lojas:
    Xloja = model_df[model_df.cod_loja == loja][:3]
    Xloja = Xloja[colunas_treino]
    Xloja.month = pd.Series([1,2,3]).values
    Xloja.quarter = pd.Series([1,1,1]).values
    Xloja.year = pd.Series([2018,2018,2018]).values
    yhat = rf.predict(Xloja[colunas_treino])
    preds.append((loja, yhat))